### Imports and constants

In [1]:
import os
import pandas as pd
import numpy as np
import yaml

SEEDS = [18, 45, 25]

NATURAL = [
    "cifar100",
    "caltech101",
    "dtd",
    "flowers102",
    "pet",
    "svhn",
    "sun397",
]
SPECIALIZED = [
    "camelyon",
    "eurosat",
    "resisc45",
    "retinopathy",
]
STRUCTURED = [
    "clevr_count",
    "clevr_distance",
    "dmlab",
    "kitti",
    "dsprites_position",
    "dsprites_orientation",
    "smallnorb_azimuth",
    "smallnorb_elevation",
]

VTAB_DATASETS = NATURAL + SPECIALIZED + STRUCTURED

FGVC_DATASETS = [
    "cub",
    "nabirds",
    "oxfordflower",
    "stanforddogs",
    "stanfordcars",
]

### Helper functions

In [2]:
def print_vtab_mean_std(results_df):
    natural_avg = sum([results_df["mean"].loc[d] for d in NATURAL]) / len(NATURAL)
    specialized_avg = sum([results_df["mean"].loc[d] for d in SPECIALIZED]) / len(SPECIALIZED)
    structured_avg = sum([results_df["mean"].loc[d] for d in STRUCTURED]) / len(STRUCTURED)
    natural_std = sum([results_df["std"].loc[d]**2 for d in NATURAL])**(1/2) / len(NATURAL)
    specialized_std = sum([results_df["std"].loc[d]**2 for d in SPECIALIZED])**(1/2) / len(SPECIALIZED)
    structured_std = sum([results_df["std"].loc[d]**2 for d in STRUCTURED])**(1/2) / len(STRUCTURED)
    overall_avg = (natural_avg + specialized_avg + structured_avg) / 3
    overall_std = (natural_std**2 + specialized_std**2 + structured_std**2)**(1/2) / 3

    print(f"Natural Avg: {natural_avg * 100:.1f} +- {natural_std * 100:.1f}")
    print(f"Specialized Avg: {specialized_avg * 100:.1f} +- {specialized_std * 100:.1f}")
    print(f"Structured Avg: {structured_avg * 100:.1f} +- {structured_std * 100:.1f}")
    print(f"Global Avg: {overall_avg * 100:.1f} +- {overall_std * 100:.1f}")

    results_list = (
        [results_df["mean"].loc[d] for d in NATURAL] + [natural_avg] +
        [results_df["mean"].loc[d] for d in SPECIALIZED] + [specialized_avg] +
        [results_df["mean"].loc[d] for d in STRUCTURED] + [structured_avg] +
        [overall_avg]
    )
    stds_list = (
        [results_df["std"].loc[d] for d in NATURAL] + [natural_std] +
        [results_df["std"].loc[d] for d in SPECIALIZED] + [specialized_std] +
        [results_df["std"].loc[d] for d in STRUCTURED] + [structured_std] +
        [overall_std]
    )

    print(" & ".join(map(lambda x: f"{100*x:.1f}", results_list)), "\\\\")
    print(" & ".join(map(lambda x: f"{100*x:.2f}", stds_list)), "\\\\")


def convert_df(eval_df, datasets):
    results_df = pd.DataFrame(index=datasets, columns=[f"seed={s}" for s in SEEDS] + ["mean", "std"])

    # convert data frame
    for dataset in datasets:
        accs = {}
        for seed in SEEDS:
            accs[f"seed={seed}"] = eval_df[eval_df.dataset == dataset].query(f"seed == {seed}")["accuracy"].item()
        row = {f"seed={s}": accs[f"seed={s}"] for s in SEEDS}
        accs_array = np.array(list(accs.values()))
        row["mean"] = np.mean(accs_array)
        row["std"] = np.std(accs_array)
        results_df.loc[dataset] = pd.Series(row)
    
    return results_df


def build_df_from_dir(basedir, datasets):
    results_df = pd.DataFrame(index=datasets, columns=[f"seed={s}" for s in SEEDS] + ["mean", "std"])
    total_params = 0

    for dataset in datasets:
        accs = {f"seed={s}": np.nan for s in SEEDS}
        path = os.path.join(basedir, dataset, f"seed={SEEDS[0]}/logs/hparams.yaml")
        with open(path, "r") as f:
            hparams = yaml.load(f, Loader=yaml.loader.SafeLoader)
            total_params += hparams["trainable_params"]
        for seed in SEEDS:
            path = os.path.join(basedir, dataset, f"seed={seed}/logs/metrics.csv")
            if not os.path.exists(path):
                continue
            df = pd.read_csv(path, sep=",")
            if not "val/accuracy" in df.keys():
                continue
            idx = df["val/accuracy"].last_valid_index()
            accs[f"seed={seed}"] = df.at[idx, "val/accuracy"]

        # only update complete runs, e.g., all SEEDS present
        if np.any(accs):
            row = {f"seed={s}": accs[f"seed={s}"] for s in SEEDS}
            accs_array = np.array(list(accs.values()))
            row["mean"] = np.mean(accs_array)
            row["std"] = np.std(accs_array)
            results_df.loc[dataset] = pd.Series(row)

    return results_df, total_params

# VTAB

## Get results from train.py logs

In [ ]:
BASEDIR = "output/vtab/adapter_plus_dim8"

results_df, total_params = build_df_from_dir(BASEDIR, VTAB_DATASETS)

print(results_df)
print_vtab_mean_std(results_df)
print(f"Average parameters: {total_params / 19 / 1e6:.2f}")


## Get results from eval.py log

In [ ]:
BASEDIR = "output/vtab/adapter_plus_dim8"

eval_df = pd.read_csv(os.path.join(BASEDIR, "eval.log"), sep="\t", names=["dataset", "seed", "accuracy"])
results_df = convert_df(eval_df, VTAB_DATASETS)

print(results_df)
print_vtab_mean_std(results_df)

# FGVC

## Get results from train.py logs

In [ ]:
BASEDIR = "output/fgvc/adapter_plus_dim1-32"

results_df, total_params = build_df_from_dir(BASEDIR, FGVC_DATASETS)
print(results_df)

avg = sum(results_df["mean"]) / 5
for d in FGVC_DATASETS:
    print(f"{d}: {results_df['mean'].loc[d] * 100:.1f}")
print(f"Average: {avg * 100:.1f}")
print(f"Average parameters: {total_params / 5 / 1e6:.2f}")


## Get results from eval.py log

In [ ]:
BASEDIR = "./output/fgvc/adapter_plus_dim1-32"

eval_df = pd.read_csv(os.path.join(BASEDIR, "eval.log"), sep="\t", names=["dataset", "seed", "accuracy"])
results_df = convert_df(eval_df, FGVC_DATASETS)
print(results_df)

avg = sum(results_df["mean"]) / 5
for d in FGVC_DATASETS:
    print(f"{d}: {results_df['mean'].loc[d] * 100:.1f}")
print(f"Average: {avg * 100:.1f}")